In [117]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor 

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [118]:
# Инициализируем все известные генераторы случаынйх чисел / Setting all known random seeds
my_code = "Романенков АА"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

os.environ['PYTHONHASHSEED']=str(my_seed)

random.seed(my_seed)

np.random.seed(my_seed)

In [119]:
# Читаем данные из файла
train_data = pd.read_csv("datasets/rus_stocks_funds_train.csv")

In [120]:
train_data.head()

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,7.210000e+09,2.870000e+09,110809.6875,3190.000000,9.870000,0.27
1,1.060000e+10,1.130000e+09,318000.0000,28230.000000,13.910000,0.30
2,2.249000e+10,1.027000e+10,329570.0000,61044.948718,4.080000,0.25
3,4.995200e+08,1.964200e+08,59380.0000,81.000000,2.910000,1.31
4,9.520000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.35


In [121]:
#pairplot(train_data)

In [122]:
# Определим размер валидационной выборки
val_size = round(0.1*len(train_data))
print(val_size)

33


In [123]:
# Создадим обучающую и валидационную выборки
random_state = my_seed
train, val = train_test_split(train_data, test_size=val_size, random_state=random_state)
print(len(train), len(val))

296 33


In [124]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
y_column = 'Debt/Assets'
x_columns = [e for e in train_data.columns if e != y_column]

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), x_columns)], remainder='passthrough')

ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['Total issued ordinary shares',
                                  'Number of shares available for sale',
                                  'Number of employees',
                                  'Number of shareholders', 'Price/Profit'])])

In [125]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_val = pd.DataFrame(ct.transform(val))

In [126]:
# Устанавливаем названия столбцов
column_names = list(x_columns) + [y_column]
sc_train.columns = column_names
sc_val.columns = column_names

In [127]:
sc_train

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,0.000021,0.023885,0.231278,0.000017,0.044349,0.58
1,0.000066,0.049084,0.231278,0.149645,0.060552,0.84
2,0.000009,0.023860,0.015184,0.149645,0.046720,0.07
3,0.000657,0.049084,0.231278,0.149645,0.023316,0.12
4,0.000309,0.024024,0.670055,0.069197,0.056029,0.30
...,...,...,...,...,...,...
291,0.000646,0.025647,0.991105,0.111928,0.012690,0.18
292,0.000013,0.023854,0.021199,0.149645,0.060464,0.42
293,0.000065,0.024067,0.231278,0.098469,0.033986,0.06
294,0.004723,0.027820,0.231278,0.692601,0.000000,0.27


In [128]:
# Отберем необходимые параметры
x_train = sc_train[x_columns]
x_val = sc_val[x_columns]

y_train = (sc_train[y_column].values).flatten()
y_val = (sc_val[y_column].values).flatten()

In [129]:
# Создадим простую модель логистической регрессии
model = TransformedTargetRegressor(regressor=LinearRegression(), transformer=MinMaxScaler())

In [130]:
# Обучим модель
model.fit(x_train, y_train)

TransformedTargetRegressor(regressor=LinearRegression(),
                           transformer=MinMaxScaler())

In [131]:
# Проверим работу обученной модели на валидационной выборке
pred_val = model.predict(x_val)
mse = mean_squared_error(y_val, pred_val)
print(mse)

0.035279220756680794


In [113]:
pred_val

array([0.3673852 , 0.32675925, 0.34466556, 0.36738608, 0.21677972,
       0.17203877, 0.37841594, 0.15247592, 0.32635946, 0.34277981,
       0.32765926, 0.3682282 , 0.32697375, 0.32762464, 0.32760804,
       0.3274215 , 0.28248171, 0.32360754, 0.21365489, 0.36436908,
       0.34208999, 0.36001352, 0.35987137, 0.32467061, 0.37175494,
       0.32834151, 0.32683739, 0.32580448, 0.34943548, 0.40148306,
       0.32743181, 0.34146935, 0.32502214, 0.36854678, 0.32667185,
       0.37987494, 0.32861006, 0.23988007, 0.36452733, 0.32586619,
       0.32618913, 0.36666974, 0.32614572, 0.37065091, 0.39322214,
       0.32944811, 0.39706163, 0.32550219, 0.37829666])

In [114]:
test = pd.read_csv("datasets/rus_stocks_funds_test.csv")

In [115]:
test[y_column] = 0.0

In [116]:
test

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,3.644700e+11,1.017600e+11,5840.0000,306350.000000,7.860000,0.0
1,6.968900e+08,4.339700e+08,110809.6875,39210.000000,5.670000,0.0
2,1.580000e+07,3.880000e+06,110809.6875,29.000000,7.060000,0.0
3,1.295000e+08,3.573000e+07,110809.6875,25.000000,14.940000,0.0
4,2.400000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.0
...,...,...,...,...,...,...
77,1.528600e+08,1.309960e+11,110809.6875,61044.948718,6.270000,0.0
78,1.295000e+08,2.634000e+07,110809.6875,25.000000,15.525191,0.0
79,2.246000e+10,0.000000e+00,281550.0000,61044.948718,6.340000,0.0
80,1.295000e+08,3.371000e+07,110809.6875,25.000000,6.320000,0.0


In [93]:
sc_test = pd.DataFrame(ct.transform(test))
sc_test.columns = column_names

In [94]:
x_test = sc_test[x_columns]

In [95]:
test[y_column] = model.predict(x_test)

In [96]:
test.head()

,Total issued ordinary shares,Number of shares available for sale,Number of employees,Number of shareholders,Price/Profit,Debt/Assets
0,3.644700e+11,1.017600e+11,5840.0000,306350.000000,7.860000,0.215082
1,6.968900e+08,4.339700e+08,110809.6875,39210.000000,5.670000,0.347217
2,1.580000e+07,3.880000e+06,110809.6875,29.000000,7.060000,0.373613
3,1.295000e+08,3.573000e+07,110809.6875,25.000000,14.940000,0.370804
4,2.400000e+09,1.309960e+11,110809.6875,61044.948718,15.525191,0.330024


In [97]:
test.to_csv('task5_results/rus_stocks_funds_result.csv', index=False)